In [22]:
from pathlib import Path
import requests
from tqdm.auto import tqdm
import tarfile

# Download and Untar Data

In [2]:
WD_PATH = Path(".")
DATA_PATH = WD_PATH / "Data"

In [3]:
if not DATA_PATH.exists():
    DATA_PATH.mkdir()

In [44]:
train_url = "http://imagenet.stanford.edu/internal/car196/cars_train.tgz"
test_url = "http://imagenet.stanford.edu/internal/car196/cars_test.tgz"
devkit_url = "https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz"

In [8]:
train_url.split('/')[-1]

'cars_train.tgz'

In [18]:
def dl(url, dest_dir):
    assert isinstance(dest_dir, Path), "dest_dir must be a Path object"
    filename = url.split('/')[-1]
    file_path = dest_dir / filename
    if not file_path.exists():
        with open(f'{dest_dir}/{filename}', 'wb') as f:
            response = requests.get(url, stream=True)
            total = int(response.headers.get('content-length'))
            with tqdm(total=total, unit='B', unit_scale=True, desc=filename) as pbar:
                for data in response.iter_content(chunk_size=1024*1024):
                    f.write(data)
                    pbar.update(1024*1024)
    else:
        return file_path
    return file_path

In [19]:
train_tar_path = dl(train_url, DATA_PATH)

In [20]:
test_tar_path = dl(test_url, DATA_PATH)

In [45]:
devkit_tar_path = dl(devkit_url, DATA_PATH)

In [53]:
def untar_tgz(tgz_path, dest_dir):
    assert isinstance(tgz_path, Path), "tgz_path must be a Path object"
    assert tgz_path.exists(), "tgz_path does not exists"
    assert tgz_path.suffix == ".tgz", "tgz_path is not a .tgz file"
    if tgz_path.stem == "car_devkit":
        final_path = dest_dir / "devkit"
    else:
        final_path = dest_dir / tgz_path.stem
    if not final_path.exists():
        tar = tarfile.open(tgz_path, 'r:gz')
        tar.extractall(dest_dir)
    return final_path

In [54]:
train_path = untar_tgz(train_tar_path, DATA_PATH)

In [55]:
test_path = untar_tgz(test_tar_path, DATA_PATH)

In [56]:
devkit_path = untar_tgz(devkit_tar_path, DATA_PATH)

# Explore

In [57]:
!ls {str(devkit_path.resolve())}

cars_meta.mat	     cars_train_annos.mat  README.txt
cars_test_annos.mat  eval_train.m	   train_perfect_preds.txt
